<h1 align="center">Implementing the Transformer Encoder from Scratch in TensorFlow and Keras</h1>

Data Scientist.: Dr.Eddy Giusepe Chirinos Isidro

Tendo visto como implementar a [atenção escalada do produto escalar](https://machinelearningmastery.com/how-to-implement-scaled-dot-product-attention-from-scratch-in-tensorflow-and-keras) e integrá-la dentro da [atenção](https://machinelearningmastery.com/how-to-implement-multi-head-attention-from-scratch-in-tensorflow-and-keras) multicabeçal do modelo Transformer, vamos progredir um passo adiante em direção à implementação de um modelo Transformer completo aplicando seu `codificador`. Nosso objetivo final continua sendo aplicar o modelo completo ao Processamento de Linguagem Natural (`NLP`).

Neste script, aprenderemos a implementar o `codificador Transformer` do zero no `TensorFlow` e no `Keras`. Ao final deste script, saberemos:

* As camadas que fazem parte do codificador Transformer.

* Como implementar o codificador Transformer do zero. 

# Implementando o Transformer Encoder do zero


## A Rede Neural Feed-Forward Totalmente Conectada e a Normalização de Camadas

Vamos começar criando classes para as camadas `Feed Forward` e `Add` & `Norm` que são mostradas no diagrama do script anterior.

`Vaswani et al.` nos dizem que a rede `feed-forward` totalmente conectada consiste em `duas transformações lineares` com uma ativação `ReLU` entre elas. A primeira transformação linear produz uma saída de dimensionalidade, $d_{ff}= 2048$, enquanto a segunda transformação linear produz uma saída de dimensionalidade, $d_{model}= 512$.

Para isso, vamos primeiro criar a classe `FeedForward` que herda da classe `Layer` base no `Keras` e inicializar as camadas `densas` e a `ativação do ReLU`:

In [ ]:
class FeedForward(Layer):
    def __init__(self, d_ff, d_model, **kwargs):
        super(FeedForward, self).__init__(**kwargs)
        self.fully_connected1 = Dense(d_ff)  # First fully connected layer
        self.fully_connected2 = Dense(d_model)  # Second fully connected layer
        self.activation = ReLU()  # ReLU activation layer
        ...

Acrescentaremos a ela o método de classe, `call()`, que recebe uma entrada e a passa pelas duas camadas totalmente conectadas com ativação `ReLU`, retornando uma saída de dimensionalidade igual a $512$:

In [ ]:
...
def call(self, x):
    # The input is passed into the two fully-connected layers, with a ReLU in between
    x_fc1 = self.fully_connected1(x)
 
    return self.fully_connected2(self.activation(x_fc1))

O próximo passo é criar outra classe, `AddNormalization`, que também herda da classe `Layer` base no Keras e inicializar uma camada de normalização de Layer:

In [ ]:
class AddNormalization(Layer):
    def __init__(self, **kwargs):
        super(AddNormalization, self).__init__(**kwargs)
        self.layer_norm = LayerNormalization()  # Layer normalization layer
        ...

Nela, inclua o seguinte método de classe que soma as entradas e saídas de sua subcamada, que recebe como entradas, e aplica a normalização da camada ao resultado:

In [ ]:
...
def call(self, x, sublayer_x):
    # The sublayer input and output need to be of the same shape to be summed
    add = x + sublayer_x
 
    # Apply layer normalization to the sum
    return self.layer_norm(add)

## A Camada do Codificador (The Encoder Layer)

Em seguida, você implementará a camada do codificador (`Encoder`), que o codificador do Transformer replicará de forma idêntica $N$ vezes. 

Para isso, vamos criar a classe, `EncoderLayer`, e inicializar todas as sub-camadas que a compõem:

In [ ]:
class EncoderLayer(Layer):
    def __init__(self, h, d_k, d_v, d_model, d_ff, rate, **kwargs):
        super(EncoderLayer, self).__init__(**kwargs)
        self.multihead_attention = MultiHeadAttention(h, d_k, d_v, d_model)
        self.dropout1 = Dropout(rate)
        self.add_norm1 = AddNormalization()
        self.feed_forward = FeedForward(d_ff, d_model)
        self.dropout2 = Dropout(rate)
        self.add_norm2 = AddNormalization()
        ...

Aqui, você pode perceber que inicializou instâncias das classes `FeedForward` e `AddNormalization`, que acabou de criar na seção anterior, e atribuiu sua saída às respectivas variáveis, `feed_forward` e `add_norm(1 and 2)`. A camada `Dropout` é autoexplicativa, onde `rate` define a frequência na qual as unidades de entrada são definidas como $0$. Você criou a classe `MultiHeadAttention` em um [tutorial anterior](https://machinelearningmastery.com/how-to-implement-multi-head-attention-from-scratch-in-tensorflow-and-keras) e, se salvou o código em um script Python separado, não se esqueça `import` ele. Salve ele em um script Python chamado `multihead_attention.py` e, por esse motivo, preciso incluir a linha de código de `multihead_attention` import `MultiHeadAttention`.

Vamos agora criar o método de classe, `call()`, que implementa todas as subcamadas do codificador:

In [2]:
...
def call(self, x, padding_mask, training):
    # Multi-head attention layer
    multihead_output = self.multihead_attention(x, x, x, padding_mask)
    # Expected output shape = (batch_size, sequence_length, d_model)
 
    # Add in a dropout layer
    multihead_output = self.dropout1(multihead_output, training=training)
 
    # Followed by an Add & Norm layer
    addnorm_output = self.add_norm1(x, multihead_output)
    # Expected output shape = (batch_size, sequence_length, d_model)
 
    # Followed by a fully connected layer
    feedforward_output = self.feed_forward(addnorm_output)
    # Expected output shape = (batch_size, sequence_length, d_model)
 
    # Add in another dropout layer
    feedforward_output = self.dropout2(feedforward_output, training=training)
 
    # Followed by another Add & Norm layer
    return self.add_norm2(addnorm_output, feedforward_output)

Além dos dados de entrada, o método `call()` também pode receber uma máscara de preenchimento (padding mask). Como um breve lembrete do que foi dito em um [tutorial anterior](https://machinelearningmastery.com/how-to-implement-scaled-dot-product-attention-from-scratch-in-tensorflow-and-keras), a máscara de preenchimento (padding mask) é necessária para impedir que o preenchimento de zero na sequência de entrada seja processado junto com os valores de entrada reais. 

O mesmo método de classe pode receber um  sinalizador `training` que, quando definido como `True`, só aplicará as camadas de Dropout durante o treinamento.

## O codificador do transformador (The Transformer Encoder)

A última etapa é criar uma classe para o `codificador do Transformer`, que deve se chamar `Encoder`:

In [37]:
class Encoder(Layer):
    def __init__(self, vocab_size, sequence_length, h, d_k, d_v, d_model, d_ff, n, rate, **kwargs):
        super(Encoder, self).__init__(**kwargs)
        self.pos_encoding = PositionEmbeddingFixedWeights(sequence_length, vocab_size, d_model)
        self.dropout = Dropout(rate)
        self.encoder_layer = [EncoderLayer(h, d_k, d_v, d_model, d_ff, rate) for _ in range(n)]
        ...

O codificador do Transformer recebe uma sequência de entrada depois que ela passou por um processo de Embedding de palavras e codificação posicional. Para calcular a codificação posicional, vamos usar a classe  `PositionEmbeddingFixedWeights` descrita por Mehreen Saeed [neste tutorial](https://machinelearningmastery.com/the-transformer-positional-encoding-layer-in-keras-part-2/). 

Como você fez da mesma forma nas seções anteriores, aqui, você também criará um método de classe, `call()`, que aplica `word Embedding` e a codificação posicional à sequência de entrada e alimenta o resulta para $N$ camadas do codificador:

In [ ]:
...
def call(self, input_sentence, padding_mask, training):
    # Generate the positional encoding
    pos_encoding_output = self.pos_encoding(input_sentence)
    # Expected output shape = (batch_size, sequence_length, d_model)
 
    # Add in a dropout layer
    x = self.dropout(pos_encoding_output, training=training)
 
    # Pass on the positional encoded values to each encoder layer
    for i, layer in enumerate(self.encoder_layer):
        x = layer(x, padding_mask, training)
 
    return x

A listagem de código para o `codificador Transformer` completo é a seguinte:

In [1]:
from tensorflow.keras.layers import LayerNormalization, Layer, Dense, ReLU, Dropout
from multihead_attention import MultiHeadAttention
from positional_encoding import PositionEmbeddingFixedWeights
 
# Implementing the Add & Norm Layer
class AddNormalization(Layer):
    def __init__(self, **kwargs):
        super(AddNormalization, self).__init__(**kwargs)
        self.layer_norm = LayerNormalization()  # Layer normalization layer
 
    def call(self, x, sublayer_x):
        # The sublayer input and output need to be of the same shape to be summed
        add = x + sublayer_x
 
        # Apply layer normalization to the sum
        return self.layer_norm(add)
 
# Implementing the Feed-Forward Layer
class FeedForward(Layer):
    def __init__(self, d_ff, d_model, **kwargs):
        super(FeedForward, self).__init__(**kwargs)
        self.fully_connected1 = Dense(d_ff)  # First fully connected layer
        self.fully_connected2 = Dense(d_model)  # Second fully connected layer
        self.activation = ReLU()  # ReLU activation layer
 
    def call(self, x):
        # The input is passed into the two fully-connected layers, with a ReLU in between
        x_fc1 = self.fully_connected1(x)
 
        return self.fully_connected2(self.activation(x_fc1))
 
# Implementing the Encoder Layer
class EncoderLayer(Layer):
    def __init__(self, h, d_k, d_v, d_model, d_ff, rate, **kwargs):
        super(EncoderLayer, self).__init__(**kwargs)
        self.multihead_attention = MultiHeadAttention(h, d_k, d_v, d_model)
        self.dropout1 = Dropout(rate)
        self.add_norm1 = AddNormalization()
        self.feed_forward = FeedForward(d_ff, d_model)
        self.dropout2 = Dropout(rate)
        self.add_norm2 = AddNormalization()
 
    def call(self, x, padding_mask, training):
        # Multi-head attention layer
        multihead_output = self.multihead_attention(x, x, x, padding_mask)
        # Expected output shape = (batch_size, sequence_length, d_model)
 
        # Add in a dropout layer
        multihead_output = self.dropout1(multihead_output, training=training)
 
        # Followed by an Add & Norm layer
        addnorm_output = self.add_norm1(x, multihead_output)
        # Expected output shape = (batch_size, sequence_length, d_model)
 
        # Followed by a fully connected layer
        feedforward_output = self.feed_forward(addnorm_output)
        # Expected output shape = (batch_size, sequence_length, d_model)
 
        # Add in another dropout layer
        feedforward_output = self.dropout2(feedforward_output, training=training)
 
        # Followed by another Add & Norm layer
        return self.add_norm2(addnorm_output, feedforward_output)
 
# Implementing the Encoder
class Encoder(Layer):
    def __init__(self, vocab_size, sequence_length, h, d_k, d_v, d_model, d_ff, n, rate, **kwargs):
        super(Encoder, self).__init__(**kwargs)
        self.pos_encoding = PositionEmbeddingFixedWeights(sequence_length, vocab_size, d_model)
        self.dropout = Dropout(rate)
        self.encoder_layer = [EncoderLayer(h, d_k, d_v, d_model, d_ff, rate) for _ in range(n)]
 
    def call(self, input_sentence, padding_mask, training):
        # Generate the positional encoding
        pos_encoding_output = self.pos_encoding(input_sentence)
        # Expected output shape = (batch_size, sequence_length, d_model)
 
        # Add in a dropout layer
        x = self.dropout(pos_encoding_output, training=training)
 
        # Pass on the positional encoded values to each encoder layer
        for i, layer in enumerate(self.encoder_layer):
            x = layer(x, padding_mask, training)
 
        return x

2022-11-23 00:31:38.449926: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-23 00:31:38.701177: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-23 00:31:38.756907: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-23 00:31:39.984973: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

# Testando o código

Você trabalhará com os valores de parâmetros especificados no artigo [Attention Is All You Need](https://arxiv.org/abs/1706.03762), de `Vaswani et al. (2017)`:

In [ ]:
h = 8  # Number of self-attention heads
d_k = 64  # Dimensionality of the linearly projected queries and keys
d_v = 64  # Dimensionality of the linearly projected values
d_ff = 2048  # Dimensionality of the inner fully connected layer
d_model = 512  # Dimensionality of the model sub-layers' outputs
n = 6  # Number of layers in the encoder stack
 
batch_size = 64  # Batch size from the training process
dropout_rate = 0.1  # Frequency of dropping the input units in the dropout layers
...

Quanto à sequência de entrada, você trabalhará com dados fictícios (dummy) por enquanto até chegar ao estágio de [Treinamento do modelo Transformer completo](https://machinelearningmastery.com/training-the-transformer-model) em um tutorial separado, momento em que você usará frases (setences) reais:

In [ ]:
...
enc_vocab_size = 20 # Vocabulary size for the encoder
input_seq_length = 5  # Maximum length of the input sequence
 
input_seq = random.random((batch_size, input_seq_length))
...

Em seguida, você criará uma nova instância da classe `Encoder`, atribuindo sua saída à variável `encoder`, subsequentemente alimentando os argumentos de entrada e imprimindo o resultado. Você definirá o argumento de máscara de preenchimento `None` por enquanto, mas retornará a ele quando implementar o `modelo Transformer completo`:

In [ ]:
...
encoder = Encoder(enc_vocab_size, input_seq_length, h, d_k, d_v, d_model, d_ff, n, dropout_rate)
print(encoder(input_seq, None, True))

<font color="orange">Juntar tudo produz a seguinte listagem de código:</font>

In [3]:
import numpy as np
from numpy import random
 
enc_vocab_size = 20 # Vocabulary size for the encoder
input_seq_length = 5  # Maximum length of the input sequence
h = 8  # Number of self-attention heads
d_k = 64  # Dimensionality of the linearly projected queries and keys
d_v = 64  # Dimensionality of the linearly projected values
d_ff = 2048  # Dimensionality of the inner fully connected layer
d_model = 512  # Dimensionality of the model sub-layers' outputs
n = 6  # Number of layers in the encoder stack
 
batch_size = 64  # Batch size from the training process
dropout_rate = 0.1  # Frequency of dropping the input units in the dropout layers
 
input_seq = random.random((batch_size, input_seq_length))
 
encoder = Encoder(enc_vocab_size, input_seq_length, h, d_k, d_v, d_model, d_ff, n, dropout_rate)
print(encoder(input_seq, None, True))

tf.Tensor(
[[[ 0.37026757  0.34895554  1.797568   ... -0.76706195 -0.28200462
    0.84410375]
  [ 0.7366137  -0.16808432  1.4786632  ... -0.879607   -0.58513445
    1.4018302 ]
  [-0.79449517 -0.03571527  1.2775862  ...  0.25089097 -0.40966454
    1.5021054 ]
  [-0.94086015 -0.55134034  0.8518615  ... -0.54215664 -0.46473953
    1.1561706 ]
  [-0.43845513 -0.06749729  0.74106365 ...  0.02672507 -0.38886154
    0.82621366]]

 [[-0.114237    0.43645588  1.1501135  ... -0.29895872 -0.02372005
    0.7629448 ]
  [-0.3223182  -0.5568473   1.8577188  ...  0.35099304 -0.26219663
    1.576568  ]
  [-1.3828976  -0.34884807  1.7991527  ...  0.1850249  -0.676788
    1.2908013 ]
  [-0.30661488 -1.0183655   2.0475793  ... -0.9062627  -0.61956036
    0.6877462 ]
  [ 0.28439015 -0.79692     1.3233557  ...  0.03901152 -0.04809393
    1.043814  ]]

 [[ 0.7505544  -0.86961806  1.3310057  ... -0.03101212 -1.3336269
    2.1856549 ]
  [ 0.3137339  -0.72647506  1.9560454  ... -0.6425798  -0.8615262
    1.465

A execução desse código produz uma saída de forma `(batch size, sequence length, model dimensionality)`. Observe que você provavelmente verá uma saída diferente devido à inicialização aleatória da sequência de entrada e aos valores dos parâmetros das camadas densas. 